In [3]:
import numpy as np
import matplotlib as mpl
import pylab as pl
import pickle
import pyekfmm as fmm
import pandas as pd
from scipy import interpolate
import matplotlib as mpl
from tqdm import tqdm
import pickle

with open('allpaths21.txt', 'rb') as file:
    allpaths = pickle.load(file)
with open('times棋盘21.txt', 'rb') as file:
    alltimes = pickle.load(file)
with open('使用1845索引.txt', 'rb') as file:
    suoyin1845 = pickle.load(file)
with open('使用台站名.txt', 'rb') as file:
    ddd = pickle.load(file)
#with open('index.txt', 'rb') as file:
    #index = pickle.load(file)
#print(len(alltimes))

a = np.full((21,9,6),5000)
a=np.nan_to_num(a, 5000)
a[a<0]=5000
array = a
mandu = 1.0 / array

nx=21
dx=0.1
ny=9
dy=0.1
nz=6
dz=0.1

data1 = pd.read_csv('./Sensors2.csv',header=None)
data2 = pd.read_csv('./events4.csv',header=None)

font_path = r'./Times New Roman.ttf'   
mpl.font_manager.fontManager.addfont(font_path)
mpl.rcParams['font.family'] = 'Times New Roman'
flag=0
m=-1
for w in range (1656):
    if w % 100 == 0:
        print(w)
    distance = [[[[]for _ in range(6)] for _ in range(9)] for _ in range(21)]
    alltime = [[[[] for _ in range(6)] for _ in range(9)] for _ in range(21)]
    for j in ddd[w]:
        m=m+1
        i=suoyin1845[w]
        x=data2.iloc[i].to_numpy()[0]/1000/39*21
        y=data2.iloc[i].to_numpy()[1]/1000/39*21
        z=data2.iloc[i].to_numpy()[2]/100/5
        vxyz=array.astype(np.float32)
        t=fmm.eikonal(vxyz.flatten(order='F'),xyz=np.array([x,y,z]),ax=[0,dx,nx],ay=[0,dy,ny],az=[0,dz,nz],order=2);
        time=t.reshape(nx,ny,nz,order='F');
        time=time*1000
        receiverx=data1.iloc[int(j-1)].to_numpy()[0]/39*21
        receivery=data1.iloc[int(j-1)].to_numpy()[1]/39*21
        receiverz=data1.iloc[int(j-1)].to_numpy()[2]/5
        tx,ty,tz = np.gradient(time)
        paths,nrays=stream3d(-tx,-ty, -tz, receiverx, receivery, receiverz, step=0.1, maxvert=10000)
        transposed_arr = paths.T
        pathss = transposed_arr.reshape(-1, 3)
        ttt1=time[int(receiverx)][int(receivery)][int(receiverz)]
        path=pathss
        ttt=alltimes[m]-ttt1
        if abs(ttt)>1:
            flag+=1
            continue
        #if ttt<0.0001:
            #flag+=1
        x0, y0,z0, x1, y1,z1 = 0, 0,0, 21, 9,6
        xgrid = np.linspace(x0, x1, 22)
        ygrid = np.linspace(y0, y1, 10)
        zgrid = np.linspace(z0, z1, 7)
        x = path[:, 0]
        y = path[:, 1]
        z = path[:, 2]
        t = np.arange(len(x))
        idx_grid, idx_t = np.where((xgrid[:, None] - x[None, :-1]) * (xgrid[:, None] - x[None, 1:]) <= 0)
        tx = idx_t + (xgrid[idx_grid] - x[idx_t]) / (x[idx_t+1] - x[idx_t])

        idx_grid, idx_t = np.where((ygrid[:, None] - y[None, :-1]) * (ygrid[:, None] - y[None, 1:]) <= 0)
        ty = idx_t + (ygrid[idx_grid] - y[idx_t]) / (y[idx_t+1] - y[idx_t])

        idx_grid, idx_t = np.where((zgrid[:, None] - z[None, :-1]) * (zgrid[:, None] - z[None, 1:]) <= 0)
        tz = idx_t + (zgrid[idx_grid] - z[idx_t]) / (z[idx_t+1] - z[idx_t])

        t2 = np.sort(np.r_[t, tx, tx, ty, ty,tz,tz])

        x2 = np.interp(t2, t, x)
        y2 = np.interp(t2, t, y)
        z2 = np.interp(t2, t, z)

        loc = np.where(np.diff(t2) == 0)[0] + 1

        xlist = np.split(x2, loc)
        ylist = np.split(y2, loc)
        zlist = np.split(z2, loc)

        points = []
        for i in range(len(xlist)):
            sublist_points = []
            for j in range(len(xlist[i])):
                point = [xlist[i][j], ylist[i][j], zlist[i][j]]
                sublist_points.append(point)
            points.append(sublist_points)
        total_distance_sum = 0
        total_distance_array = []
    # 循环迭代每个列表
        for coordinates in points:
        # 将列表转换为 NumPy 数组
            coordinates_array = np.array(coordinates)
        # 计算差值
            differences = np.diff(coordinates_array, axis=0)
        # 计算距离
            distances = np.linalg.norm(differences, axis=1)
        # 计算总距离
            total_distance = np.sum(distances)
            total_distance_array.append(total_distance)
            total_distance_sum += total_distance

        ratio_array = np.divide(total_distance_array,total_distance_sum )
        ratio_array[np.isnan(ratio_array)]=0
        everytime=ratio_array*ttt
    #print('总长度：',total_distance_sum)
    #print('每段长度:',total_distance_array)
    #print('每段时间：',everytime)
    #print(result)
        #distance = [[[[]for _ in range(30)] for _ in range(18)] for _ in range(39)]
        #alltime = [[[[] for _ in range(30)] for _ in range(18)] for _ in range(39)]
        #print(k)
        if len(points)>=2:
            for l in range(len(points)):
                if len(points[l])>1:
                    a=int(points[l][1][0])
                    b=int(points[l][1][1])
                    c=int(points[l][1][2])
                    if a<21 and b<9 and c<6:
                        distance[a][b][c].append(total_distance_array[l])
                        alltime[a][b][c].append(everytime[l])
    #print(distance)
    #print(alltime)
        sumdistance = [[[[]for _ in range(6)] for _ in range(9)] for _ in range(21)]
        sumalltime = [[[[] for _ in range(6)] for _ in range(9)] for _ in range(21)]
        for i in range(len(distance)):
            for j in range(len(distance[i])):
                for k in range(len(distance[i][j])):
                    sumdistance[i][j][k]=sum(distance[i][j][k])
                    sumalltime[i][j][k]=sum(alltime[i][j][k])
    #print(sumdistance)
    #print(sumalltime)
        result = []
        for i in range(len( sumdistance)):
            layer = []
            for j in range(len( sumdistance[i])):
                row = []
                for k in range(len( sumdistance[i][j])):
                    if sumdistance[i][j][k] != 0:
                        quotient = sumalltime[i][j][k] / (sumdistance[i][j][k]*100)
                        if np.isnan(quotient)==True:
                            row.append(0)
                        else:
                            row.append(quotient)
                    else:
                        row.append(0)
                layer.append(row)
            result.append(layer)
        arr = np.array(result)
        mandu=mandu+arr*0.01
        array=1.0 / mandu
        array[np.isnan(array)]=5000
        array[array<0]=5000
        #if flag==20000:
            #break
velocity=1.0 / mandu
print(velocity[velocity!=5000].shape)
np.save('qipan1224.npy', velocity)

0


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:90: RuntimeWarning: invalid value encountered in true_divide


100
200
300


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:87: RuntimeWarning: invalid value encountered in true_divide


400
500
600
700
800


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:84: RuntimeWarning: invalid value encountered in true_divide


900
1000
1100
1200
1300
1400
1500
1600
(429,)


In [2]:
import numpy as np
import matplotlib as mpl
import pylab as pl
import pickle
import pyekfmm as fmm
import pandas as pd
from scipy import interpolate
import matplotlib as mpl
from tqdm import tqdm
import pickle

with open('allpaths21.txt', 'rb') as file:
    allpaths = pickle.load(file)
with open('times棋盘21.txt', 'rb') as file:
    alltimes = pickle.load(file)
with open('使用1845索引.txt', 'rb') as file:
    suoyin1845 = pickle.load(file)
with open('使用台站名.txt', 'rb') as file:
    ddd = pickle.load(file)
#with open('index.txt', 'rb') as file:
    #index = pickle.load(file)
#print(len(alltimes))

a = np.full((21,9,6),5000)
a=np.nan_to_num(a, 5000)
a[a<0]=5000
array = a
mandu = 1.0 / array

nx=21
dx=0.1
ny=9
dy=0.1
nz=6
dz=0.1

data1 = pd.read_csv('./Sensors2.csv',header=None)
data2 = pd.read_csv('./events4.csv',header=None)

font_path = r'./Times New Roman.ttf'   
mpl.font_manager.fontManager.addfont(font_path)
mpl.rcParams['font.family'] = 'Times New Roman'
flag=0
m=-1
for w in range (1656):
    if w % 100 == 0:
        print(w)
    distance = [[[[]for _ in range(6)] for _ in range(9)] for _ in range(21)]
    alltime = [[[[] for _ in range(6)] for _ in range(9)] for _ in range(21)]
    for j in ddd[w]:
        m=m+1
        i=suoyin1845[w]
        x=data2.iloc[i].to_numpy()[0]/1000/39*21
        y=data2.iloc[i].to_numpy()[1]/1000/39*21
        z=data2.iloc[i].to_numpy()[2]/100/5
        vxyz=array.astype(np.float32)
        t=fmm.eikonal(vxyz.flatten(order='F'),xyz=np.array([x,y,z]),ax=[0,dx,nx],ay=[0,dy,ny],az=[0,dz,nz],order=2);
        time=t.reshape(nx,ny,nz,order='F');
        time=time*1000
        receiverx=data1.iloc[int(j-1)].to_numpy()[0]/39*21
        receivery=data1.iloc[int(j-1)].to_numpy()[1]/39*21
        receiverz=data1.iloc[int(j-1)].to_numpy()[2]/5
        tx,ty,tz = np.gradient(time)
        paths,nrays=stream3d(-tx,-ty, -tz, receiverx, receivery, receiverz, step=0.1, maxvert=10000)
        transposed_arr = paths.T
        pathss = transposed_arr.reshape(-1, 3)
        path=pathss
        #ttt=alltimes[m]-ttt1
        '''if abs(ttt)>1:
            flag+=1
            continue'''
        #if ttt<0.0001:
            #flag+=1
        x0, y0,z0, x1, y1,z1 = 0, 0,0, 21, 9,6
        xgrid = np.linspace(x0, x1, 22)
        ygrid = np.linspace(y0, y1, 10)
        zgrid = np.linspace(z0, z1, 7)
        x = path[:, 0]
        y = path[:, 1]
        z = path[:, 2]
        t = np.arange(len(x))
        idx_grid, idx_t = np.where((xgrid[:, None] - x[None, :-1]) * (xgrid[:, None] - x[None, 1:]) <= 0)
        tx = idx_t + (xgrid[idx_grid] - x[idx_t]) / (x[idx_t+1] - x[idx_t])

        idx_grid, idx_t = np.where((ygrid[:, None] - y[None, :-1]) * (ygrid[:, None] - y[None, 1:]) <= 0)
        ty = idx_t + (ygrid[idx_grid] - y[idx_t]) / (y[idx_t+1] - y[idx_t])

        idx_grid, idx_t = np.where((zgrid[:, None] - z[None, :-1]) * (zgrid[:, None] - z[None, 1:]) <= 0)
        tz = idx_t + (zgrid[idx_grid] - z[idx_t]) / (z[idx_t+1] - z[idx_t])

        t2 = np.sort(np.r_[t, tx, tx, ty, ty,tz,tz])

        x2 = np.interp(t2, t, x)
        y2 = np.interp(t2, t, y)
        z2 = np.interp(t2, t, z)

        loc = np.where(np.diff(t2) == 0)[0] + 1

        xlist = np.split(x2, loc)
        ylist = np.split(y2, loc)
        zlist = np.split(z2, loc)

        points = []
        for i in range(len(xlist)):
            sublist_points = []
            for j in range(len(xlist[i])):
                point = [xlist[i][j], ylist[i][j], zlist[i][j]]
                sublist_points.append(point)
            points.append(sublist_points)
        total_distance_sum = 0
        total_distance_array = []
    # 循环迭代每个列表
        for coordinates in points:
        # 将列表转换为 NumPy 数组
            coordinates_array = np.array(coordinates)
        # 计算差值
            differences = np.diff(coordinates_array, axis=0)
        # 计算距离
            distances = np.linalg.norm(differences, axis=1)
        # 计算总距离
            total_distance = np.sum(distances)
            total_distance_array.append(total_distance)
            total_distance_sum += total_distance

        ratio_array = np.divide(total_distance_array,total_distance_sum )
        ratio_array[np.isnan(ratio_array)]=0
        #everytime=ratio_array*ttt
    #print('总长度：',total_distance_sum)
    #print('每段长度:',total_distance_array)
    #print('每段时间：',everytime)
    #print(result)
        #distance = [[[[]for _ in range(30)] for _ in range(18)] for _ in range(39)]
        #alltime = [[[[] for _ in range(30)] for _ in range(18)] for _ in range(39)]
        #print(k)
        my_array = np.zeros((nx, ny, nz))
        tt=np.zeros((nx, ny, nz))
        if len(points)>=2:
            for l in range(len(points)):
                if len(points[l])>1:
                    a=int(points[l][1][0])
                    b=int(points[l][1][1])
                    c=int(points[l][1][2])
                    if a<21 and b<9 and c<6:
                        my_array[a][b][c]=total_distance_array[l]
                        #alltime[a][b][c].append(everytime[l])
                        velocity=array
                        vv=velocity[a][b][c]
                        tt[a][b][c]=my_array[a][b][c]/vv*100
        totalSum = np.sum(tt)
        ttt=alltimes[m]-totalSum
        everytime=ratio_array*ttt
        if len(points)>=2:
            for l in range(len(points)):
                if len(points[l])>1:
                    if np.isnan(points[l][1][0])==True:
                        continue
                    else:
                        a=int(points[l][1][0])
                        b=int(points[l][1][1])
                        c=int(points[l][1][2])
                    if a<21 and b<9 and c<6:
                        distance[a][b][c].append(total_distance_array[l])
                        alltime[a][b][c].append(everytime[l])
    #print(alltime)
        sumdistance = [[[[]for _ in range(6)] for _ in range(9)] for _ in range(21)]
        sumalltime = [[[[] for _ in range(6)] for _ in range(9)] for _ in range(21)]
        for i in range(len(distance)):
            for j in range(len(distance[i])):
                for k in range(len(distance[i][j])):
                    sumdistance[i][j][k]=sum(distance[i][j][k])
                    sumalltime[i][j][k]=sum(alltime[i][j][k])
    #print(sumdistance)
    #print(sumalltime)
        result = []
        for i in range(len( sumdistance)):
            layer = []
            for j in range(len( sumdistance[i])):
                row = []
                for k in range(len( sumdistance[i][j])):
                    if sumdistance[i][j][k] != 0:
                        quotient = sumalltime[i][j][k] / (sumdistance[i][j][k]*100)
                        if np.isnan(quotient)==True:
                            row.append(0)
                        else:
                            row.append(quotient)
                    else:
                        row.append(0)
                layer.append(row)
            result.append(layer)
        arr = np.array(result)
        mandu=mandu+arr*0.01
        array=1.0 / mandu
        array[np.isnan(array)]=5000
        array[array<0]=5000
        #if flag==20000:
            #break
velocity=1.0 / mandu
print(velocity[velocity!=5000].shape)
np.save('qipan路径时间.npy', velocity)

0


NameError: name 'ttt1' is not defined

In [ ]:
import numpy as np
import matplotlib as mpl
import pylab as pl
import pickle
import pyekfmm as fmm
import pandas as pd
from scipy import interpolate
import matplotlib as mpl
from tqdm import tqdm
import pickle

with open('allpaths21.txt', 'rb') as file:
    allpaths = pickle.load(file)
with open('times棋盘21.txt', 'rb') as file:
    alltimes = pickle.load(file)
with open('使用1845索引.txt', 'rb') as file:
    suoyin1845 = pickle.load(file)
with open('使用台站名.txt', 'rb') as file:
    ddd = pickle.load(file)
#with open('index.txt', 'rb') as file:
    #index = pickle.load(file)
#print(len(alltimes))

a = np.full((21,9,6),5000)
a=np.nan_to_num(a, 5000)
a[a<0]=5000
array = a
mandu = 1.0 / array

nx=21
dx=0.1
ny=9
dy=0.1
nz=6
dz=0.1

data1 = pd.read_csv('./Sensors2.csv',header=None)
data2 = pd.read_csv('./events4.csv',header=None)

font_path = r'./Times New Roman.ttf'   
mpl.font_manager.fontManager.addfont(font_path)
mpl.rcParams['font.family'] = 'Times New Roman'
flag=0
m=-1
for w in range (1656):
    if w % 100 == 0:
        print(w)
    distance = [[[[]for _ in range(6)] for _ in range(9)] for _ in range(21)]
    alltime = [[[[] for _ in range(6)] for _ in range(9)] for _ in range(21)]
    for j in ddd[w]:
        m=m+1
        i=suoyin1845[w]
        x=data2.iloc[i].to_numpy()[0]/1000/39*21
        y=data2.iloc[i].to_numpy()[1]/1000/39*21
        z=data2.iloc[i].to_numpy()[2]/100/5
        vxyz=array.astype(np.float32)
        t=fmm.eikonal(vxyz.flatten(order='F'),xyz=np.array([x,y,z]),ax=[0,dx,nx],ay=[0,dy,ny],az=[0,dz,nz],order=2);
        time=t.reshape(nx,ny,nz,order='F');
        time=time*1000
        receiverx=data1.iloc[int(j-1)].to_numpy()[0]/39*21
        receivery=data1.iloc[int(j-1)].to_numpy()[1]/39*21
        receiverz=data1.iloc[int(j-1)].to_numpy()[2]/5
        path=allpaths[m]
        #ttt=alltimes[m]-ttt1
        '''if abs(ttt)>1:
            flag+=1
            continue'''
        #if ttt<0.0001:
            #flag+=1
        x0, y0,z0, x1, y1,z1 = 0, 0,0, 21, 9,6
        xgrid = np.linspace(x0, x1, 22)
        ygrid = np.linspace(y0, y1, 10)
        zgrid = np.linspace(z0, z1, 7)
        x = path[:, 0]
        y = path[:, 1]
        z = path[:, 2]
        t = np.arange(len(x))
        idx_grid, idx_t = np.where((xgrid[:, None] - x[None, :-1]) * (xgrid[:, None] - x[None, 1:]) <= 0)
        tx = idx_t + (xgrid[idx_grid] - x[idx_t]) / (x[idx_t+1] - x[idx_t])

        idx_grid, idx_t = np.where((ygrid[:, None] - y[None, :-1]) * (ygrid[:, None] - y[None, 1:]) <= 0)
        ty = idx_t + (ygrid[idx_grid] - y[idx_t]) / (y[idx_t+1] - y[idx_t])

        idx_grid, idx_t = np.where((zgrid[:, None] - z[None, :-1]) * (zgrid[:, None] - z[None, 1:]) <= 0)
        tz = idx_t + (zgrid[idx_grid] - z[idx_t]) / (z[idx_t+1] - z[idx_t])

        t2 = np.sort(np.r_[t, tx, tx, ty, ty,tz,tz])

        x2 = np.interp(t2, t, x)
        y2 = np.interp(t2, t, y)
        z2 = np.interp(t2, t, z)

        loc = np.where(np.diff(t2) == 0)[0] + 1

        xlist = np.split(x2, loc)
        ylist = np.split(y2, loc)
        zlist = np.split(z2, loc)

        points = []
        for i in range(len(xlist)):
            sublist_points = []
            for j in range(len(xlist[i])):
                point = [xlist[i][j], ylist[i][j], zlist[i][j]]
                sublist_points.append(point)
            points.append(sublist_points)
        total_distance_sum = 0
        total_distance_array = []
    # 循环迭代每个列表
        for coordinates in points:
        # 将列表转换为 NumPy 数组
            coordinates_array = np.array(coordinates)
        # 计算差值
            differences = np.diff(coordinates_array, axis=0)
        # 计算距离
            distances = np.linalg.norm(differences, axis=1)
        # 计算总距离
            total_distance = np.sum(distances)
            total_distance_array.append(total_distance)
            total_distance_sum += total_distance

        ratio_array = np.divide(total_distance_array,total_distance_sum )
        ratio_array[np.isnan(ratio_array)]=0
        #everytime=ratio_array*ttt
    #print('总长度：',total_distance_sum)
    #print('每段长度:',total_distance_array)
    #print('每段时间：',everytime)
    #print(result)
        #distance = [[[[]for _ in range(30)] for _ in range(18)] for _ in range(39)]
        #alltime = [[[[] for _ in range(30)] for _ in range(18)] for _ in range(39)]
        #print(k)
        my_array = np.zeros((nx, ny, nz))
        tt=np.zeros((nx, ny, nz))
        if len(points)>=2:
            for l in range(len(points)):
                if len(points[l])>1:
                    a=int(points[l][1][0])
                    b=int(points[l][1][1])
                    c=int(points[l][1][2])
                    if a<21 and b<9 and c<6:
                        my_array[a][b][c]=total_distance_array[l]
                        #alltime[a][b][c].append(everytime[l])
                        velocity=array
                        vv=velocity[a][b][c]
                        tt[a][b][c]=my_array[a][b][c]/vv*100
        totalSum = np.sum(tt)
        ttt=alltimes[m]-totalSum
        everytime=ratio_array*ttt
        if len(points)>=2:
            for l in range(len(points)):
                if len(points[l])>1:
                    if np.isnan(points[l][1][0])==True:
                        continue
                    else:
                        a=int(points[l][1][0])
                        b=int(points[l][1][1])
                        c=int(points[l][1][2])
                    if a<21 and b<9 and c<6:
                        distance[a][b][c].append(total_distance_array[l])
                        alltime[a][b][c].append(everytime[l])
    #print(alltime)
        sumdistance = [[[[]for _ in range(6)] for _ in range(9)] for _ in range(21)]
        sumalltime = [[[[] for _ in range(6)] for _ in range(9)] for _ in range(21)]
        for i in range(len(distance)):
            for j in range(len(distance[i])):
                for k in range(len(distance[i][j])):
                    sumdistance[i][j][k]=sum(distance[i][j][k])
                    sumalltime[i][j][k]=sum(alltime[i][j][k])
    #print(sumdistance)
    #print(sumalltime)
        result = []
        for i in range(len( sumdistance)):
            layer = []
            for j in range(len( sumdistance[i])):
                row = []
                for k in range(len( sumdistance[i][j])):
                    if sumdistance[i][j][k] != 0:
                        quotient = sumalltime[i][j][k] / (sumdistance[i][j][k]*100)
                        if np.isnan(quotient)==True:
                            row.append(0)
                        else:
                            row.append(quotient)
                    else:
                        row.append(0)
                layer.append(row)
            result.append(layer)
        arr = np.array(result)
        mandu=mandu+arr*0.01
        array=1.0 / mandu
        array[np.isnan(array)]=5000
        array[array<0]=5000
        #if flag==20000:
            #break
velocity=1.0 / mandu
print(velocity[velocity!=5000].shape)
np.save('qipan时间.npy', velocity)

In [1]:
import pyekfmm as fmm
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import interpolate
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.font_manager import FontProperties
import matplotlib as mpl
import time as ti
from tqdm import tqdm
import pickle
from joblib import Parallel, delayed
def traceStreamUVW (vgrid, ugrid, wgrid, ydim, xdim,  zdim, sy, sx, sz, step, maxvert):
	"""
	traceStreamUVW: 3D streamline
	
	INPUT
	vgrid:   	derivative of traveltime in y
	ugrid:   	derivative of traveltime in x
	wgrid:   	derivative of traveltime in z
	
	OUTPUT  
	
	Copyright (C) 2023 The University of Texas at Austin
	Copyright (C) 2023 Yangkang Chen
	
	This program is free software: you can redistribute it and/or modify
	it under the terms of the GNU General Public License as published
	by the Free Software Foundation, either version 3 of the License, or
	any later version.
	
	This program is distributed in the hope that it will be useful,
	but WITHOUT ANY WARRANTY; without even the implied warranty of
	MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
	GNU General Public License for more details: http://www.gnu.org/licenses/
	
	References   
	[1] Chen et al., 2023
		
	 DEMO
	 demos/test_xxx.py
	"""
	
	numverts=0;
	
	x=sx-1.0;y=sy-1.0;z=sz-1.0;
	verts=np.zeros([3*maxvert,1])

	while 1:
		if x<0:
			x=0;
		if y<0:
			y=0;
		if z<0:
			z=0;
		if (x<0 or x>xdim or y<0 or y>ydim or z<0 or z>zdim or numverts>=maxvert) :
			#print("First break");
			break;

		ix=int(np.floor(x))
		iy=int(np.floor(y))
		iz=int(np.floor(z))
		
		if ix == xdim-1:
			ix=ix-1;
			
		if iy == ydim-1:
			iy=iy-1;

		if iz == zdim-1:
			iz=iz-1;
		
		xfrac=x-ix;
		yfrac=y-iy;
		zfrac=z-iz;
		#weights for linear interpolation
		a=(1-xfrac)*(1-yfrac)*(1-zfrac);
		b=(  xfrac)*(1-yfrac)*(1-zfrac);
		c=(1-xfrac)*(  yfrac)*(1-zfrac);
		d=(  xfrac)*(  yfrac)*(1-zfrac);
		e=(1-xfrac)*(1-yfrac)*(  zfrac);
		f=(  xfrac)*(1-yfrac)*(  zfrac);
		g=(1-xfrac)*(  yfrac)*(  zfrac);
		h=(  xfrac)*(  yfrac)*(  zfrac);
		
		verts[3*numverts + 0] = y+1;
		verts[3*numverts + 1] = x+1;
		verts[3*numverts + 2] = z+1;
		#if already been here, done
		if numverts>=2:
			if verts[3*numverts] == verts[3*(numverts-2)] and verts[3*numverts+1] == verts[3*(numverts-2)+1] and verts[3*numverts+2] == verts[3*(numverts-2)+2]:
				numverts=numverts+1;
				#print("Second break");
				break;
		
		numverts=numverts+1;
		ui = ugrid[iy  +ydim*ix +xdim*ydim*iz]*a + ugrid[iy  +ydim*(ix+1) +xdim*ydim*iz]*b + ugrid[iy+1+ydim*ix+xdim*ydim*iz]*c + ugrid[iy+1+ydim*(ix+1)+xdim*ydim*iz]*d + ugrid[iy  +ydim*ix +xdim*ydim*(iz+1)]*e + ugrid[iy  +ydim*(ix+1) +xdim*ydim*(iz+1)]*f + ugrid[iy+1+ydim*ix+xdim*ydim*(iz+1)]*g + ugrid[iy+1+ydim*(ix+1)+xdim*ydim*(iz+1)]*h;
		vi = vgrid[iy  +ydim*ix +xdim*ydim*iz]*a + vgrid[iy  +ydim*(ix+1) +xdim*ydim*iz]*b + vgrid[iy+1+ydim*ix+xdim*ydim*iz]*c + vgrid[iy+1+ydim*(ix+1)+xdim*ydim*iz]*d + vgrid[iy  +ydim*ix +xdim*ydim*(iz+1)]*e + vgrid[iy  +ydim*(ix+1) +xdim*ydim*(iz+1)]*f + vgrid[iy+1+ydim*ix+xdim*ydim*(iz+1)]*g + vgrid[iy+1+ydim*(ix+1)+xdim*ydim*(iz+1)]*h;
		wi = wgrid[iy  +ydim*ix +xdim*ydim*iz]*a + wgrid[iy  +ydim*(ix+1) +xdim*ydim*iz]*b + wgrid[iy+1+ydim*ix+xdim*ydim*iz]*c + wgrid[iy+1+ydim*(ix+1)+xdim*ydim*iz]*d + wgrid[iy  +ydim*ix +xdim*ydim*(iz+1)]*e + wgrid[iy  +ydim*(ix+1) +xdim*ydim*(iz+1)]*f + wgrid[iy+1+ydim*ix+xdim*ydim*(iz+1)]*g + wgrid[iy+1+ydim*(ix+1)+xdim*ydim*(iz+1)]*h;
	
		#calculate step size, if 0, done
		if abs(ui) > abs(vi):
			imax=abs(ui);
		else:
			imax=abs(vi);
			
		if abs(wi)>imax:
			imax=abs(wi);
			
		if imax==0:
			print("Third break");
			break;

		imax=step/imax;
		ui = ui*imax;
		vi = vi*imax;
		wi = wi*imax;
		
		#update the current position
		x = x+ui;
		y = y+vi;
		z = z+wi;

	verts=verts[0:3*numverts] 
	
	return verts,numverts

def stream3d(u,v, w, sx, sy, sz, step=0.1, maxvert=10000):
	"""
	stream3d: draw 3D stream lines along the steepest descent direction
	
	INPUT
	u:   	derivative of traveltime in x
	v:   	derivative of traveltime in z
	w:   	derivative of traveltime in y
	
	OUTPUT  
	
	Copyright (C) 2023 The University of Texas at Austin
	Copyright (C) 2023 Yangkang Chen
	
	This program is free software: you can redistribute it and/or modify
	it under the terms of the GNU General Public License as published
	by the Free Software Foundation, either version 3 of the License, or
	any later version.
	
	This program is distributed in the hope that it will be useful,
	but WITHOUT ANY WARRANTY; without even the implied warranty of
	MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
	GNU General Public License for more details: http://www.gnu.org/licenses/
	
	References   
	[1] Chen et al.
		
	 DEMO
	 demos/test_xxx.py
	"""
	
	xSize=u.shape[0]
	ySize=u.shape[1]
	zSize=u.shape[2]
	[verts, numverts] = traceStreamUVW  (u.flatten(order='F'),v.flatten(order='F'),w.flatten(order='F'), xSize, ySize, zSize, sx, sy, sz, step, maxvert);
	verts=verts.reshape(3,numverts,order='F');
	
	return verts,numverts
	

    

In [19]:
import numpy as np
import matplotlib as mpl
import pylab as pl
import pickle
import pyekfmm as fmm
import pandas as pd
from scipy import interpolate
import matplotlib as mpl
from tqdm import tqdm
import pickle

with open('allpaths21.txt', 'rb') as file:
    allpaths = pickle.load(file)
with open('times棋盘21.txt', 'rb') as file:
    alltimes = pickle.load(file)
with open('使用1845索引.txt', 'rb') as file:
    suoyin1845 = pickle.load(file)
with open('使用台站名.txt', 'rb') as file:
    ddd = pickle.load(file)
#with open('index.txt', 'rb') as file:
    #index = pickle.load(file)
#print(len(alltimes))

a = np.full((21,9,6),5000)
a=np.nan_to_num(a, 5000)
a[a<0]=5000
array = a
mandu = 1.0 / array

nx=21
dx=0.1
ny=9
dy=0.1
nz=6
dz=0.1

data1 = pd.read_csv('./Sensors2.csv',header=None)
data2 = pd.read_csv('./events4.csv',header=None)

font_path = r'./Times New Roman.ttf'   
mpl.font_manager.fontManager.addfont(font_path)
mpl.rcParams['font.family'] = 'Times New Roman'
flag=0
m=-1
for w in range (1656):
    if w % 100 == 0:
        print(w)
    distance = [[[[]for _ in range(6)] for _ in range(9)] for _ in range(21)]
    alltime = [[[[] for _ in range(6)] for _ in range(9)] for _ in range(21)]
    for j in ddd[w]:
        m=m+1
        i=suoyin1845[w]
        x=data2.iloc[i].to_numpy()[0]/1000/39*21
        y=data2.iloc[i].to_numpy()[1]/1000/39*21
        z=data2.iloc[i].to_numpy()[2]/100/5
        vxyz=array.astype(np.float32)
        t=fmm.eikonal(vxyz.flatten(order='F'),xyz=np.array([x,y,z]),ax=[0,dx,nx],ay=[0,dy,ny],az=[0,dz,nz],order=2);
        time=t.reshape(nx,ny,nz,order='F');
        time=time*1000
        tx,ty,tz = np.gradient(time)
        receiverx=data1.iloc[int(j-1)].to_numpy()[0]/39*21
        receivery=data1.iloc[int(j-1)].to_numpy()[1]/39*21
        receiverz=data1.iloc[int(j-1)].to_numpy()[2]/5
        ttt1=time[int(receiverx)][int(receivery)][int(receiverz)]
        paths,nrays=stream3d(-tx,-ty, -tz, receiverx, receivery, receiverz, step=0.1, maxvert=10000)
        transposed_arr = paths.T
        path1 = transposed_arr.reshape(-1, 3)
        path=path1
        ttt=alltimes[m]-ttt1
        if abs(ttt)>1:
            flag+=1
            continue
        #if ttt<0.0001:
            #flag+=1
        x0, y0,z0, x1, y1,z1 = 0, 0,0, 21, 9,6
        xgrid = np.linspace(x0, x1, 22)
        ygrid = np.linspace(y0, y1, 10)
        zgrid = np.linspace(z0, z1, 7)
        x = path[:, 0]
        y = path[:, 1]
        z = path[:, 2]
        t = np.arange(len(x))
        idx_grid, idx_t = np.where((xgrid[:, None] - x[None, :-1]) * (xgrid[:, None] - x[None, 1:]) <= 0)
        tx = idx_t + (xgrid[idx_grid] - x[idx_t]) / (x[idx_t+1] - x[idx_t])

        idx_grid, idx_t = np.where((ygrid[:, None] - y[None, :-1]) * (ygrid[:, None] - y[None, 1:]) <= 0)
        ty = idx_t + (ygrid[idx_grid] - y[idx_t]) / (y[idx_t+1] - y[idx_t])

        idx_grid, idx_t = np.where((zgrid[:, None] - z[None, :-1]) * (zgrid[:, None] - z[None, 1:]) <= 0)
        tz = idx_t + (zgrid[idx_grid] - z[idx_t]) / (z[idx_t+1] - z[idx_t])

        t2 = np.sort(np.r_[t, tx, tx, ty, ty,tz,tz])

        x2 = np.interp(t2, t, x)
        y2 = np.interp(t2, t, y)
        z2 = np.interp(t2, t, z)

        loc = np.where(np.diff(t2) == 0)[0] + 1

        xlist = np.split(x2, loc)
        ylist = np.split(y2, loc)
        zlist = np.split(z2, loc)

        points = []
        for i in range(len(xlist)):
            sublist_points = []
            for j in range(len(xlist[i])):
                point = [xlist[i][j], ylist[i][j], zlist[i][j]]
                sublist_points.append(point)
            points.append(sublist_points)
        total_distance_sum = 0
        total_distance_array = []
    # 循环迭代每个列表
        for coordinates in points:
        # 将列表转换为 NumPy 数组
            coordinates_array = np.array(coordinates)
        # 计算差值
            differences = np.diff(coordinates_array, axis=0)
        # 计算距离
            distances = np.linalg.norm(differences, axis=1)
        # 计算总距离
            total_distance = np.sum(distances)
            total_distance_array.append(total_distance)
            total_distance_sum += total_distance

        ratio_array = np.divide(total_distance_array,total_distance_sum )
        ratio_array[np.isnan(ratio_array)]=0
        everytime=ratio_array*ttt
    #print('总长度：',total_distance_sum)
    #print('每段长度:',total_distance_array)
    #print('每段时间：',everytime)
    #print(result)
        #distance = [[[[]for _ in range(30)] for _ in range(18)] for _ in range(39)]
        #alltime = [[[[] for _ in range(30)] for _ in range(18)] for _ in range(39)]
        #print(k)
        if len(points)>=2:
            for l in range(len(points)):
                if len(points[l])>1:
                    a=int(points[l][1][0])
                    b=int(points[l][1][1])
                    c=int(points[l][1][2])
                    if a<21 and b<9 and c<6:
                        distance[a][b][c].append(total_distance_array[l])
                        alltime[a][b][c].append(everytime[l])
    #print(distance)
    #print(alltime)
        sumdistance = [[[[]for _ in range(6)] for _ in range(9)] for _ in range(21)]
        sumalltime = [[[[] for _ in range(6)] for _ in range(9)] for _ in range(21)]
        for i in range(len(distance)):
            for j in range(len(distance[i])):
                for k in range(len(distance[i][j])):
                    sumdistance[i][j][k]=sum(distance[i][j][k])
                    sumalltime[i][j][k]=sum(alltime[i][j][k])
    #print(sumdistance)
    #print(sumalltime)
        result = []
        for i in range(len( sumdistance)):
            layer = []
            for j in range(len( sumdistance[i])):
                row = []
                for k in range(len( sumdistance[i][j])):
                    if sumdistance[i][j][k] != 0:
                        quotient = sumalltime[i][j][k] / (sumdistance[i][j][k]*100)
                        if np.isnan(quotient)==True:
                            row.append(0)
                        else:
                            row.append(quotient)
                    else:
                        row.append(0)
                layer.append(row)
            result.append(layer)
        arr = np.array(result)
        mandu=mandu+arr*0.05
        array=1.0 / mandu
        array[np.isnan(array)]=5000
        array[array<0]=5000
        #if flag==20000:
            #break
velocity=1.0 / mandu
print(velocity[velocity!=5000].shape)
np.save('qipan212121.npy', velocity)

0


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:90: RuntimeWarning: invalid value encountered in true_divide


100
200
300


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:87: RuntimeWarning: invalid value encountered in true_divide


400
500
600
700
800


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:84: RuntimeWarning: invalid value encountered in true_divide


900
1000
1100
1200
1300
1400
1500
1600
(424,)
